# YOLOv5 Training and Conversion to RKNN - Team 5990 TRIGON
<img src ="https://avatars.githubusercontent.com/u/45858082?s=280&v=4" />

In [ ]:
%cd {root_path}
import os
root_path = os.getcwd()

!git clone https://github.com/airockchip/yolov5/
%cd yolov5
!git checkout d25a07534c14f44296f9444bab2aa5c601cdaaab

!pip install -qr requirements.txt

## Downloading the dataset
Input your Roboflow API key below. It can be obtained [here](https://app.roboflow.com/settings/api).
You can use your own dataset, the rest of the notebook should work with any number of classes, as long as the project is of "object detection" type.

In [ ]:
!mkdir {root_path}/datasets
%cd {root_path}/datasets

!pip install roboflow -q

from roboflow import Roboflow
rf = Roboflow(api_key="api_key")
project = rf.workspace("trigon5990").project("conesandcubes")
dataset = project.version(6).download("yolov5")


In [ ]:
import os
import re
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
        print("Wrote successfully to " + line)

@register_line_cell_magic
def replaceAllInFile(line, cell):
    filename = line.strip()
    replacements = eval(cell)  # Assuming input is a valid Python expression
    with open(filename, 'r') as f:
        file_content = f.read()
    for replaced, with_this in replacements:
        file_content = re.sub(replaced, with_this, file_content)
    with open(filename, 'w') as f:
        f.write(file_content)
    print(f"Replaced successfully in {filename}")

In [ ]:
%cat {dataset.location}/data.yaml

#### This is needed because the default format of the directories in roboflow datasets is kinda broken:

In [ ]:
%%replaceAllInFile {dataset.location}/data.yaml

[
    ('test: ..', 'test: ' + dataset.location),
    ('train: ', 'train: ' + root_path + '/datasets/'),
    ('val: ', 'val: ' + root_path + '/datasets/'),
]


In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
print(f"num_classes: {num_classes}")
%cd {root_path}/yolov5

## Training
You can adjust the following settings:
- model: one of [yolov5n, yolov5s, yolov5m, yolov5l, yolov5x]
- image_size: The input size of the images fed to the model. Should be a multiple of 32.
- batch: Number of samples per epoch. You should set this to the highest number possible without the training taking too much memory (it would crash if that happens, which is ok, just lower the number and try again)
- epochs: How many iterations to train for
- patience: After how many epochs without improvement to stop the training

In [ ]:
%cd {root_path}/yolov5
!wandb disabled
model = "yolov5n"
image_size = 640
!python train.py --img {image_size} --batch 256 --epochs 700 --patience 50 --data {dataset.location}/data.yaml --cfg {model}.yaml --weights '' --device 0 --cache {"--single-cls" if num_classes == '1' else ""}

In [ ]:
latest_modified_time = 0
latest = None

for foldername, subfolders, filenames in os.walk(root_path):
    for filename in filenames:
        if filename == "best.pt":
            file_path = os.path.join(foldername, filename)
            modified_time = os.path.getmtime(file_path)
            if modified_time > latest_modified_time:
                latest_modified_time = modified_time
                latest = file_path
print(latest, latest_modified_time)

## Exporting to ONNX
This is an intermediate step between the PyTorch model and the RKNN model.

In [ ]:
%cd {root_path}/yolov5
!python export.py --rknpu --weight {latest}

In [ ]:
ex_path = '.'.join(latest.split('.')[:-1] + ['onnx'])
print(ex_path)

### Installing RKNN Toolkit 2

In [ ]:
!wget https://github.com/rockchip-linux/rknn-toolkit2/raw/2c2d03def0c0908c86985b8190e973976ecec74c/rknn-toolkit2/packages/rknn_toolkit2-1.6.0+81f21f4d-cp310-cp310-linux_x86_64.whl
!pip install ./rknn_toolkit2-1.6.0+81f21f4d-cp310-cp310-linux_x86_64.whl

In [ ]:
%cd {root_path}
!git clone https://github.com/airockchip/rknn_model_zoo/
%cd rknn_model_zoo
!git checkout eaa94d6f57ca553d493bf3bd7399a070452d2774
%cd examples/yolov5/python

In [ ]:
%%writefile imgs.txt
imgs/1.jpg
imgs/2.jpg
imgs/3.jpg
imgs/4.jpg
imgs/5.jpg
imgs/6.jpg
imgs/7.jpg
imgs/8.jpg
imgs/9.jpg
imgs/10.jpg
imgs/11.jpg
imgs/12.jpg
imgs/13.jpg
imgs/14.jpg
imgs/15.jpg
imgs/16.jpg
imgs/17.jpg
imgs/18.jpg
imgs/19.jpg
imgs/20.jpg


In [ ]:
import os
import shutil
import random
import glob

def copy_and_rename_images(source_folder, destination_folder, n):
    if not os.path.exists(source_folder):
        print(f"Source folder '{source_folder}' does not exist.")
        return
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    image_files = glob.glob(os.path.join(source_folder, '*.jpg'))
    selected_images = random.sample(image_files, min(n, len(image_files)))
    for i, image_path in enumerate(selected_images, start=1):
        destination_path = os.path.join(destination_folder, f'{i}.jpg')
        shutil.copy(image_path, destination_path)
    print(f"{min(n, len(image_files))} random images copied from '{source_folder}' to '{destination_folder}' and renamed.")
copy_and_rename_images(dataset.location+"/test/images" , "imgs", 20)

In [ ]:
%%replaceAllInFile {root_path}/rknn_model_zoo/examples/yolov5/python/convert.py
[('../../../datasets/COCO/coco_subset_20.txt', 'imgs.txt')]

## Quantization
Here you choose whether to perform quantization, which makes the model lighter and faster, by converting all 32/16 bit floates in the model into 8 bit ints, which costs performance.

In [ ]:
to_quantize = True # @param {type: "boolean"}

## Exporting to RKNN - Final step🎉

In [ ]:
%cd {root_path}/rknn_model_zoo/examples/yolov5/python
quant_code = "i8" if to_quantize else "fp"
output_model = f"{root_path}/{dataset.name}-{model}-{image_size}-{quant_code}.rknn"
!python convert.py {ex_path} rk3588 {quant_code} {output_model}